In [1]:
import random
import numpy as np
import tensorflow as tf
import math
import os

import matplotlib.pyplot as plt
import data_utils
#from cs231n.assignment1.cs231n.data_utils import load_CIFAR10

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
#from tensorflow.examples.tutorials.mnist import input_data as mnist_data

/home/kumar/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

# Load the raw CIFAR-10 data.
cifar10_dir = 'cs231n/assignment1/cs231n/datasets/cifar-10-batches-py'
X_train, y_train, X_test, y_test = data_utils.load_CIFAR10(cifar10_dir)
#y_train = tf.one_hot(y_tn, 10)
#y_test = tf.one_hot(y_tst, 10)

#print(type(X_train),type(y_train),type(y_test))
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()

#print(type(X_train),type(y_train),type(y_test), type(X_test))

#y_train = np.array(y_train)
#y_test = np.array(y_test)

#print(type(X_train),type(y_train),type(y_test), type(X_test))
# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

('Training data shape: ', (50000, 32, 32, 3))
('Training labels shape: ', (50000, 10))
('Test data shape: ', (10000, 32, 32, 3))
('Test labels shape: ', (10000, 10))


In [3]:
print ("Tensorflow version " + tf.__version__)

Tensorflow version 1.5.0


In [4]:
def createPlaceHolders(n_width, n_height, n_depth, n_classes):
    X = tf.placeholder(tf.float32, [None,n_width, n_height, n_depth ])
    y = tf.placeholder(tf.float32,  [None, n_classes])
    step = tf.placeholder(tf.int32)
    return X,y,step

In [5]:
def intialize_parameters(K,L,M,N,O,F):
    parameters = {}
    W1 = tf.get_variable('W1', shape = [3, 3, 3, K], initializer = tf.contrib.layers.xavier_initializer())
    B1 = tf.Variable(tf.ones([K])/10)
    
    W2 = tf.get_variable('W2', shape=[3, 3, K, L], initializer = tf.contrib.layers.xavier_initializer())
    B2 = tf.Variable(tf.ones([L])/10)
    
    W3 = tf.get_variable('W3', shape=[3, 3, L, M], initializer = tf.contrib.layers.xavier_initializer()) 
    B3 = tf.Variable(tf.ones([M])/10)
    
    W4 = tf.get_variable('W4', shape=[3, 3, M, N], initializer = tf.contrib.layers.xavier_initializer()) 
    B4 = tf.Variable(tf.ones([N])/10)
    
    W5 = tf.get_variable('W5', shape=[3, 3, N, O], initializer = tf.contrib.layers.xavier_initializer()) 
    B5 = tf.Variable(tf.ones([O])/10)
    
    W6 = tf.get_variable('W6', shape=[8 * 8 * O, F], initializer = tf.contrib.layers.xavier_initializer())
    B6 = tf.Variable(tf.ones([F])/10)
    
    W7 = tf.get_variable('W7', shape=[F, F], initializer = tf.contrib.layers.xavier_initializer())
    B7 = tf.Variable(tf.ones([F])/10)
    
    W8 = tf.get_variable('W8', shape=[F, 10], initializer = tf.contrib.layers.xavier_initializer())
    B8 = tf.Variable(tf.ones([10])/10)
    
    parameters = {"W1":W1, "B1":B1, "W2":W2, "B2":B2,"W3":W3,"B3":B3,"W4":W4, "B4":B4,"W5":W5, "B5":B5, "W6":W6, "B6":B6,
                 "W7":W7, "B7":B7, "W8":W8, "B8":B8}
    return parameters

In [6]:
def predict(parameters,X, O, n_inputs, n_classes):
    #X,y = createPlaceHolders(n_inputs, n_classes)
    #parameters = intialize_parameter(K,L,M,N)
    s = 1 # stride
    # OUTPUT : 32 * 32 * K
    conv1 = tf.nn.relu(tf.nn.conv2d(X, parameters['W1'], strides=[1,s,s,1], padding = 'SAME') + parameters['B1'])
    
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
            padding='SAME', name='pool1') 
    #max pooling
    #pool1 = tf.nn.max_pool()

    #s = 2 # stride
    # OUTPUT : 16 * 16 * L
    conv2 = tf.nn.relu(tf.nn.conv2d(pool1, parameters['W2'], strides=[1,s,s,1], padding = 'SAME') + parameters['B2'])
    
    #OUTPUT : 8 * 8 * M
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
            padding='SAME', name='pool2')
    
    #s = 1 # stride
    #OUTPUT : 8 * 8 * N
    conv3 = tf.nn.relu(tf.nn.conv2d(pool2, parameters['W3'], strides=[1,s,s,1], padding = 'SAME') + parameters['B3'])
    '''
    New code
    '''
    #OUTPUT : 8 * 8 * 'O'
    conv4 = tf.nn.relu(tf.nn.conv2d(conv3, parameters['W4'], strides=[1,s,s,1], padding = 'SAME') + parameters['B4'])
    
    #OUTPUT : 8 * 8 * 'O'
    conv5 = tf.nn.relu(tf.nn.conv2d(conv4, parameters['W5'], strides=[1,s,s,1], padding = 'SAME') + parameters['B5'])
    
    #pool3 = tf.nn.max_pool(conv5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME', name='pool3')
    ##################################################
    #falttern
    yy = tf.reshape(conv5,  [-1, 8 * 8 * O])  # shape =
    
    fc1 = tf.nn.relu(tf.matmul(yy, parameters['W6']) + parameters['B6'])
    
    fc2 = tf.nn.relu(tf.matmul(fc1, parameters['W7']) + parameters['B7'])
    
    yLogit = tf.matmul(fc2, parameters['W8']) + parameters['B8']  
    
    return yLogit
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
def find_loss(yLogit, y):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=yLogit, labels=y)
    loss = tf.reduce_mean(entropy)
    return loss

In [8]:
def find_accuracy(yLogit, y):
    prediction = tf.nn.softmax(yLogit)
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [16]:
def model():
    
    #mnist = mnist_data.read_data_sets("data", one_hot=True)
    
    n_classes = 10
    n_width   = 32
    n_height  = 32
    n_depth   = 3
    n_inputs = n_height * n_width * n_depth # total pixels
    
    # three convolution layers 
    K = 96   # First layer depth
    L = 128   # Second layer depth
    M = 256  # Third layer depth
    N = 256
    O = 128
    F = 1024 # Fully connected 

    #learning_rate = 0.001
    n_epochs = 1
    epochs = 1000
    batch_size = 64
    num_train = X_train.shape[0]
    num_test  = X_test.shape[0]
    #n_batches = int(mnist.train.num_examples/batch_size)
    
    
    X, y, step = createPlaceHolders(n_width, n_height, n_depth, n_classes)
    parameters = intialize_parameters(K, L, M, N,O,F)
    yLogit     = predict(parameters, X, O, n_inputs, n_classes)
    loss       = find_loss(yLogit, y)
    accuracy   = find_accuracy(yLogit, y)
    
    lr = 0.0001 +  tf.train.exponential_decay(0.003, step, 2000, 1/math.e)
    optimizer = tf.train.AdamOptimizer(lr).minimize(loss)
    
    
    acc = 0.0
    lo = 0.0
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    for i in xrange(epochs):
        batch = np.random.choice(num_train, batch_size, replace=True)
        #print(X_train.shape, y_train.shape)
        batch_X = X_train[batch]
        batch_y = y_train[batch]
        #print(type(X_train), type(y_train))
        #print(type(batch_X), type(batch_y))
        #print(X_train.shape, y_train.shape)
        #print(batch_X.shape, batch_y.shape)
        #batch_X, batch_y = mnist.train.next_batch(batch_size)
        a, c, _,_ = sess.run([accuracy, loss , optimizer,lr], feed_dict={X : batch_X, y : batch_y, step : i})
        print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c))
        acc = a + acc
        lo = c + lo
    print acc,lo
    print('Train accuracy is  ', acc/epochs)
    print('Train loss is  ', lo/epochs)
    acc = 0.0
    lo = 0.0
    for i in xrange(100):
        batch = np.random.choice(num_test, batch_size, replace=True)
        batch_X  = X_test[batch]
        batch_y  = y_test[batch]
        #batch_X, batch_y = mnist.test.next_batch(batch_size)
        a, c = sess.run([accuracy, loss], feed_dict={X : batch_X, y : batch_y })
        print(str(i) + ": ********* epoch " + " ********* test accuracy:" + str(a) + " test loss: " + str(c))
        acc = a + acc
        lo = c+lo
    print acc, lo
    print('Test accuracy is  ', acc/100)
    print('Test loss is  ', lo/100)
    sess.close()
    

In [17]:
tf.reset_default_graph() 
model()

0: accuracy:0.0625 loss: 10.443335
1: accuracy:0.078125 loss: 4843.8057
2: accuracy:0.109375 loss: 60.464592
3: accuracy:0.125 loss: 13.030361
4: accuracy:0.109375 loss: 3.697246
5: accuracy:0.109375 loss: 2.892477
6: accuracy:0.0625 loss: 5.6485195
7: accuracy:0.046875 loss: 3.5829885
8: accuracy:0.078125 loss: 2.6503305
9: accuracy:0.09375 loss: 3.2024364
10: accuracy:0.078125 loss: 2.3408313
11: accuracy:0.0625 loss: 2.3654873
12: accuracy:0.171875 loss: 2.3076565
13: accuracy:0.140625 loss: 2.3056645
14: accuracy:0.0625 loss: 2.8077688
15: accuracy:0.140625 loss: 2.325612
16: accuracy:0.09375 loss: 2.38283
17: accuracy:0.125 loss: 2.376186
18: accuracy:0.125 loss: 2.3233714
19: accuracy:0.046875 loss: 2.3968065
20: accuracy:0.109375 loss: 2.4136658
21: accuracy:0.046875 loss: 2.3928943
22: accuracy:0.09375 loss: 2.3963046
23: accuracy:0.0625 loss: 2.3199182
24: accuracy:0.09375 loss: 2.3307152
25: accuracy:0.125 loss: 2.3758273
26: accuracy:0.109375 loss: 2.4058218
27: accuracy:0.0

228: accuracy:0.359375 loss: 1.7908658
229: accuracy:0.28125 loss: 1.766748
230: accuracy:0.234375 loss: 2.0070002
231: accuracy:0.21875 loss: 1.9274843
232: accuracy:0.296875 loss: 2.012517
233: accuracy:0.3125 loss: 1.930645
234: accuracy:0.34375 loss: 1.8897874
235: accuracy:0.25 loss: 2.027147
236: accuracy:0.234375 loss: 1.9919751
237: accuracy:0.234375 loss: 2.0307717
238: accuracy:0.265625 loss: 1.9074578
239: accuracy:0.21875 loss: 1.9405515
240: accuracy:0.28125 loss: 1.8684604
241: accuracy:0.296875 loss: 2.051521
242: accuracy:0.125 loss: 2.5471213
243: accuracy:0.171875 loss: 2.10929
244: accuracy:0.3125 loss: 1.8296273
245: accuracy:0.234375 loss: 2.1197743
246: accuracy:0.25 loss: 1.9151626
247: accuracy:0.125 loss: 2.1968746
248: accuracy:0.296875 loss: 2.034807
249: accuracy:0.203125 loss: 2.1861005
250: accuracy:0.1875 loss: 2.1917815
251: accuracy:0.21875 loss: 1.9979202
252: accuracy:0.265625 loss: 1.9527557
253: accuracy:0.28125 loss: 1.999589
254: accuracy:0.296875

454: accuracy:0.390625 loss: 1.8271841
455: accuracy:0.25 loss: 1.9276515
456: accuracy:0.328125 loss: 1.888476
457: accuracy:0.234375 loss: 1.9487395
458: accuracy:0.3125 loss: 1.9440781
459: accuracy:0.5 loss: 1.7370532
460: accuracy:0.46875 loss: 1.5935103
461: accuracy:0.265625 loss: 1.7679147
462: accuracy:0.296875 loss: 1.9537944
463: accuracy:0.390625 loss: 1.7566227
464: accuracy:0.34375 loss: 1.8029798
465: accuracy:0.3125 loss: 1.7827708
466: accuracy:0.390625 loss: 1.649928
467: accuracy:0.21875 loss: 1.9543958
468: accuracy:0.375 loss: 1.8112974
469: accuracy:0.25 loss: 1.8429978
470: accuracy:0.328125 loss: 1.67429
471: accuracy:0.28125 loss: 1.7464154
472: accuracy:0.3125 loss: 1.9335431
473: accuracy:0.34375 loss: 1.6635543
474: accuracy:0.203125 loss: 1.9875417
475: accuracy:0.21875 loss: 1.9377445
476: accuracy:0.296875 loss: 1.8384873
477: accuracy:0.40625 loss: 1.8343444
478: accuracy:0.375 loss: 1.858696
479: accuracy:0.3125 loss: 1.8604407
480: accuracy:0.390625 lo

680: accuracy:0.40625 loss: 1.657563
681: accuracy:0.359375 loss: 1.74786
682: accuracy:0.265625 loss: 1.9149857
683: accuracy:0.390625 loss: 1.6498299
684: accuracy:0.28125 loss: 1.9867097
685: accuracy:0.390625 loss: 1.757831
686: accuracy:0.390625 loss: 1.6567796
687: accuracy:0.4375 loss: 1.6347069
688: accuracy:0.34375 loss: 1.7012842
689: accuracy:0.28125 loss: 1.7435988
690: accuracy:0.25 loss: 1.9820523
691: accuracy:0.3125 loss: 1.9028103
692: accuracy:0.359375 loss: 1.60663
693: accuracy:0.359375 loss: 1.6171573
694: accuracy:0.328125 loss: 1.7168024
695: accuracy:0.359375 loss: 1.8741368
696: accuracy:0.34375 loss: 1.8071932
697: accuracy:0.328125 loss: 1.6917275
698: accuracy:0.375 loss: 1.8209127
699: accuracy:0.375 loss: 1.8354983
700: accuracy:0.34375 loss: 1.8428261
701: accuracy:0.34375 loss: 1.621765
702: accuracy:0.359375 loss: 1.7834127
703: accuracy:0.375 loss: 1.8682716
704: accuracy:0.328125 loss: 1.685554
705: accuracy:0.265625 loss: 1.8457611
706: accuracy:0.39

903: accuracy:0.359375 loss: 1.5226858
904: accuracy:0.375 loss: 1.6463282
905: accuracy:0.453125 loss: 1.6098554
906: accuracy:0.390625 loss: 1.6371559
907: accuracy:0.453125 loss: 1.5895295
908: accuracy:0.359375 loss: 1.604965
909: accuracy:0.375 loss: 1.750766
910: accuracy:0.28125 loss: 1.8867939
911: accuracy:0.453125 loss: 1.5203328
912: accuracy:0.421875 loss: 1.764841
913: accuracy:0.359375 loss: 1.752958
914: accuracy:0.40625 loss: 1.769166
915: accuracy:0.5 loss: 1.4842038
916: accuracy:0.40625 loss: 1.5413188
917: accuracy:0.3125 loss: 1.8477292
918: accuracy:0.34375 loss: 1.8549414
919: accuracy:0.296875 loss: 1.7413505
920: accuracy:0.3125 loss: 1.9294037
921: accuracy:0.390625 loss: 1.6617683
922: accuracy:0.390625 loss: 1.5911286
923: accuracy:0.359375 loss: 1.5994691
924: accuracy:0.34375 loss: 1.7358816
925: accuracy:0.328125 loss: 1.8151941
926: accuracy:0.4375 loss: 1.6920637
927: accuracy:0.390625 loss: 1.6090798
928: accuracy:0.40625 loss: 1.7108344
929: accuracy:

73: ********* epoch  ********* test accuracy:0.390625 test loss: 1.6487644
74: ********* epoch  ********* test accuracy:0.34375 test loss: 1.6424075
75: ********* epoch  ********* test accuracy:0.40625 test loss: 1.6970112
76: ********* epoch  ********* test accuracy:0.390625 test loss: 1.7874639
77: ********* epoch  ********* test accuracy:0.375 test loss: 1.7044489
78: ********* epoch  ********* test accuracy:0.3125 test loss: 1.7884096
79: ********* epoch  ********* test accuracy:0.390625 test loss: 1.7023814
80: ********* epoch  ********* test accuracy:0.34375 test loss: 1.5755265
81: ********* epoch  ********* test accuracy:0.1875 test loss: 1.8890799
82: ********* epoch  ********* test accuracy:0.28125 test loss: 1.939431
83: ********* epoch  ********* test accuracy:0.421875 test loss: 1.5867829
84: ********* epoch  ********* test accuracy:0.296875 test loss: 1.8451083
85: ********* epoch  ********* test accuracy:0.359375 test loss: 1.8658171
86: ********* epoch  ********* test a